In [1]:
description = """The first floor contains a promethium generator and a promethium-compatible microchip.
The second floor contains a cobalt generator, a curium generator, a ruthenium generator, and a plutonium generator.
The third floor contains a cobalt-compatible microchip, a curium-compatible microchip, a ruthenium-compatible microchip, and a plutonium-compatible microchip.
The fourth floor contains nothing relevant."""

import re

floors = [re.findall("a\s.+[\.,]", floor) for floor in description.split("\n")]
#welp


i wanna do this with sat solver, where each floor needs to have rules:
- (if:) X-Mic (then) X-Gen or all other -GEN (not true) -> Allowrd (True?)
- so thats 5 rules for 4 floors
- so 4 truth tables (floors) with 10 columns each

a generated move is only valid if it both affected truth tables are valid?


In [2]:

pos= """
kM cM rM uM
kG cG rG uM
** pG pM"""
pos_iso = """
kM cM rM uM
kG cG rG uM
** pM pG"""

goal = """** PmG PmM CbG CbM CmG CmM RuG RuM PuG PuM


"""

pos = """
kM cM rM uM
kG cG rG uG
** pG pM"""

print("],[".join([f"','".join(sorted([i for i in line.split(" ")])) for i, line in enumerate(pos.split("\n"))]))

],[cM','kM','rM','uM],[cG','kG','rG','uG],[**','pG','pM


In [ ]:
from aocutils.solvers import Puzzle
from itertools import combinations

class FloorPuzzle(Puzzle):
    goal = [sorted(['**','cM','kM','rM','uM','cG','kG','rG','uG','pG','pM']),[],[],[]]

    # pos = [[],['cM','kM','rM','uM'],['cG','kG','rG','uG'],['**','pG','pM']]


    def __repr__(self):
        return "\n".join([f"F{4-i}:"+",".join(sorted([i for i in line])) for i, line in enumerate(self.pos)])
    
    def is_valid(self, floor_items):
        microchips = [item[0] for item in floor_items if item[1:]=="M"]
        generators = set(item[0] for item in floor_items if item[1:]=="G")
        return not generators or all([element in generators for element in microchips]) 
    
    def cfn(self):
        """current floor number"""
        for i, floor in enumerate(self.pos):
            if "**" in floor: return i
            
    def above_below(self, current_floor_id):
        if current_floor_id == 0: return [1]
        elif 1 <= current_floor_id <=2: return [0,1,2,3][current_floor_id-1::2]
        else: return [2]
           
    def possible_takes(self):
        items = self.items_on(self.cfn())
        return [*combinations(items,1), *combinations(items, 2)]
    
    def items_on(self, n):
        return set(self.pos[n]) - {"**"}
    
    def transfer(self, what, from_floorn:str, to_floorn):
        floors = [floor.copy() for floor in self.pos]
        floors[from_floorn]= sorted([item for item in floors[from_floorn] if not item in what])
        floors[to_floorn] = sorted(floors[to_floorn] + what)
        return floors
    
    def __iter__(self):
        current_floor = self.items_on(self.cfn())
        for combination in self.possible_takes() :
            elevator_content = set([*combination, "**"])
            if not self.is_valid(current_floor - elevator_content): continue
            for new_fn in self.above_below(self.cfn()):
                new_floor = self.items_on(new_fn)
                                
                if not self.is_valid(new_floor | elevator_content): continue
                
                # print("adding", *elevator_content, "to floor", 4-new_fn, ":", *new_floor)
                valid = FloorPuzzle(pos=self.transfer(list(elevator_content), self.cfn(), new_fn))
                yield valid

    
    
simple = FloorPuzzle([[],['cM','kM','rM','uM'],['cG','kG','rG','uG'],['**','pG','pM']])
simple


F4:
F3:cM,kM,rM,uM
F2:cG,kG,rG,uG
F1:**,pG,pM

In [21]:

simple.solve(depthFirst=False, verbose=True)

__________ i=0,c=0 Queue: 1
F4:
F3:cM,kM,rM,uM
F2:cG,kG,rG,uG
F1:**,pG,pM
__________ i=1,c=0 Queue: 2
F4:
F3:cM,kM,rM,uM
F2:cG,kG,rG,uG
F1:**,pG,pM
__________ i=2,c=2 Queue: 1
F4:
F3:cM,kM,rM,uM
F2:**,cG,kG,pG,rG,uG
F1:pM
__________ i=3,c=3 Queue: 4
F4:
F3:cM,kM,rM,uM
F2:**,cG,kG,pG,pM,rG,uG
F1:
__________ i=4,c=4 Queue: 15
F4:
F3:cM,kM,rM,uM
F2:cG,rG,uG
F1:**,kG,pG,pM
__________ i=5,c=5 Queue: 16
F4:
F3:cM,kM,rM,uM
F2:cG,kG,uG
F1:**,pG,pM,rG
__________ i=6,c=7 Queue: 16
F4:
F3:cM,kM,rM,uM
F2:cG,kG,rG
F1:**,pG,pM,uG
__________ i=7,c=9 Queue: 16
F4:
F3:cM,kM,rM,uM
F2:kG,rG,uG
F1:**,cG,pG,pM
__________ i=8,c=11 Queue: 16
F4:
F3:cM,kM,rM,uM
F2:cG,pG,pM,rG,uG
F1:**,kG
__________ i=9,c=12 Queue: 15
F4:
F3:cM,kM,rM,uM
F2:cG,kG,pG,pM,uG
F1:**,rG
__________ i=10,c=13 Queue: 14
F4:
F3:cM,kM,rM,uM
F2:cG,kG,pG,pM,rG
F1:**,uG
__________ i=11,c=14 Queue: 13
F4:
F3:**,cM,kM,pM,rM,uM
F2:cG,kG,pG,rG,uG
F1:
__________ i=12,c=15 Queue: 41
F4:
F3:cM,kM,rM,uM
F2:cG,kG,pG,rG,uG
F1:**,pM
__________ i=13,c=1

KeyboardInterrupt: 

In [ ]:
p1 = FloorPuzzle([[],['cM','kM','rM','uM'],['cG','kG','rG','uG'],['**','pG','pM']])
p2 = FloorPuzzle([[],['cM','kM','rM','uM'],['cG','kG','rG','uG'],['pG','pM', '**']])
p1.canonical() == p2.canonical()


True

In [29]:
class SimplePuzzle(FloorPuzzle):
    pos = [[],["LG"],["HG"],["**","HM","LM"]]
    goal = [["**","LG","HG","HM","LM"],[],[],[]]
    
simple = SimplePuzzle()
for i in simple:
    print(i)

F4:
F3:LG
F2:**,HG,HM
F1:LM


# IPYNB DOESNT YIELD MORE THAN ONCE😣😣😣😣😣😣😣
FAAAAAAAAAAAACKK ARE YOU KIDDING ME, the problem is that a yield in a jupyter notebook does not continue past a first iteration!!!